In [ ]:
from manim import *
import numpy as np

class PCADeepDiveScene(ThreeDScene):
    def construct(self):
        # ----------------- SETTINGS -----------------
        # Set the 3D camera orientation
        self.set_camera_orientation(phi=75*DEGREES, theta=-60*DEGREES, distance=12)
        fps = 30 # Not directly used in Manim's rendering but good to note

        # ----------------- DATASET -----------------
        # Define a 3D curve (similar to a Lissajous figure, slightly flattened z)
        def curve_func(t):
            return np.array([np.cos(t)*2, np.sin(t)*2, 0.8*np.sin(2*t)])

        # Generate points
        t_vals = np.linspace(0, 2*np.pi, 30)
        points = np.array([curve_func(t) for t in t_vals])

        # 3D axes + points
        axes = ThreeDAxes(
            x_range=[-5, 5, 1], y_range=[-5, 5, 1], z_range=[-3, 3, 1],
            x_length=8, y_length=8, z_length=6
        )
        axes_labels = axes.get_axis_labels(
            x_label=MathTex("x_1"), y_label=MathTex("x_2"), z_label=MathTex("x_3")
        )
        self.add(axes, axes_labels)
        dots = VGroup(*[Dot3D(point=pt, color=BLUE, radius=0.08) for pt in points])
        self.play(FadeIn(dots), run_time=1.0)
        self.wait(0.2)

        # ----------------- MEAN -----------------
        mean = points.mean(axis=0)
        mean_dot = Dot3D(point=mean, color=RED, radius=0.12)
        # Display the mean vector
        mean_label = MathTex(r"\bar{x}").scale(0.7).to_corner(UL).shift(RIGHT*0.5)
        mean_coords = MathTex(r"=" + f"({mean[0]:.2f}," + f"{mean[1]:.2f}," + f"{mean[2]:.2f})").scale(0.6).next_to(mean_label, RIGHT, buff=0.1)
        cov_formula = MathTex(r"\Sigma = \frac{1}{N}\sum_i (x_i-\bar{x})(x_i-\bar{x})^T").scale(0.55)
        cov_formula.next_to(mean_label, DOWN, aligned_edge=LEFT).shift(DOWN*0.1)
        self.add_fixed_in_frame_mobjects(mean_label, mean_coords, cov_formula)
        self.play(FadeIn(mean_dot), Write(mean_label), Write(mean_coords), Write(cov_formula), run_time=1.2)
        self.wait(0.5)

        # ----------------- COVARIANCE MATRIX (numeric) -----------------
        centered = points - mean
        cov = np.cov(centered.T, bias=True) # 1/N normalization (Population covariance)
        cov_rounded = np.round(cov, 3)
        cov_matrix = Matrix([[f"{v:.3f}" for v in row] for row in cov_rounded])
        cov_matrix.scale(0.65).to_corner(UR).shift(LEFT*0.3 + DOWN*0.2)
        cov_label = MathTex(r"\Sigma").scale(0.7).next_to(cov_matrix, LEFT, buff=0.2)
        self.add_fixed_in_frame_mobjects(cov_matrix, cov_label)
        self.play(FadeIn(cov_matrix), FadeIn(cov_label), run_time=0.8)
        self.wait(0.6)

        # ----------------- EIGENDECOMPOSITION -----------------
        eigvals, eigvecs = np.linalg.eigh(cov) # Get eigenvalues and eigenvectors
        # Sort by eigenvalue in descending order
        idx = np.argsort(eigvals)[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        
        # Display the decomposition equation
        eqn = MathTex(r"\Sigma = Q \Lambda Q^\top").scale(0.7).next_to(cov_matrix, LEFT, buff=0.2).shift(DOWN*0.1)
        
        # Display Lambda matrix (Eigenvalues)
        lambda_mat = np.diag(eigvals)
        lambda_disp = Matrix([[f"{v:.3f}" if i==j else "0.000" for j,vj in enumerate(lambda_mat[i])] for i in range(3)])
        lambda_disp.scale(0.6).next_to(cov_matrix, DOWN, buff=0.4).shift(LEFT*0.2)
        lambda_label = MathTex(r"\Lambda").scale(0.7).next_to(lambda_disp, UP, buff=0.1)
        
        # Display Q matrix (Eigenvectors / Principal Components)
        q_disp = Matrix([[f"{v:.3f}" for v in eigvecs[:,i]] for i in range(3)]).T # Transpose to show column vectors
        q_disp.scale(0.6).next_to(lambda_disp, LEFT, buff=0.4)
        q_label = MathTex(r"Q").scale(0.7).next_to(q_disp, UP, buff=0.1)
        
        # Display Q^T matrix (Transpose of Eigenvectors)
        qT_disp = Matrix([[f"{v:.3f}" for v in eigvecs[:,i]] for i in range(3)])
        qT_disp.scale(0.6).next_to(lambda_disp, RIGHT, buff=0.4)
        qT_label = MathTex(r"Q^\top").scale(0.7).next_to(qT_disp, UP, buff=0.1)
        
        # Animate the display of the eigendecomposition
        self.play(Transform(cov_label, eqn),
                  FadeIn(q_disp, shift=UP*0.5), FadeIn(lambda_disp, shift=UP*0.5), FadeIn(qT_disp, shift=UP*0.5),
                  FadeIn(q_label), FadeIn(lambda_label), FadeIn(qT_label),
                  run_time=1.5)
        self.wait(0.8)

        # ----------------- PRINCIPAL AXES (visual) -----------------
        # Normalize eigenvectors (they should be already normalized by eigh, but good practice)
        eigvecs = eigvecs / np.linalg.norm(eigvecs, axis=0)
        pc_arrows = VGroup()
        colors = [YELLOW, GREEN, PURPLE]
        scale_factor = 2.0 # Length proportional to sqrt(lambda)
        pc_labels = VGroup()
        
        for i in range(3):
            v = eigvecs[:, i]
            # Length is proportional to the standard deviation (sqrt(eigenvalue))
            length = np.sqrt(max(eigvals[i], 0)) * scale_factor
            start = mean - v * length
            end = mean + v * length
            arr = Arrow3D(start=start, end=end, thickness=0.03, color=colors[i], tip_length=0.2)
            pc_arrows.add(arr)
            # Label the principal component axis
            lab = MathTex(fr"\text{{PC}}_{i+1}", color=colors[i]).scale(0.5)
            lab.move_to(mean + v*(length + 0.45))
            self.add_fixed_orientation_mobjects(lab)
            pc_labels.add(lab)
            
        self.play(Create(pc_arrows), run_time=1.8)
        
        # Note on arrow length
        std_note = MathTex(r"\text{length} \propto \sqrt{\lambda_i}", color=WHITE).scale(0.55)
        std_note.to_corner(DR).shift(UP*0.2)
        self.add_fixed_in_frame_mobjects(std_note)
        self.play(FadeIn(std_note))
        self.wait(0.6)

        # ----------------- EXPLAINED VARIANCE BARS -----------------
        explained = eigvals / eigvals.sum()
        cum_explained = np.cumsum(explained)
        
        # Create bars as rectangles in fixed frame
        bars = VGroup()
        labels_bars = VGroup()
        max_h = 2.5
        
        title_bars = MathTex(r"\text{Explained Variance (\lambda_i / \sum \lambda)}").scale(0.6).to_corner(UR).shift(LEFT*2.2 + DOWN*0.2)
        self.add_fixed_in_frame_mobjects(title_bars)
        
        for i, val in enumerate(explained):
            rect = Rectangle(width=0.6, height=val*max_h, fill_opacity=0.8, fill_color=colors[i], stroke_width=1)
            # Position the bar plot
            rect.move_to(title_bars.get_bottom() + DOWN*1.5 + RIGHT*(i*0.9))
            rect.align_data_and_family(UP)
            
            num = MathTex(f"{val*100:.1f}\\%").scale(0.45)
            num.next_to(rect, DOWN, buff=0.07)
            bars.add(rect)
            labels_bars.add(num)
            
            self.add_fixed_in_frame_mobjects(rect, num)
            
        self.play(FadeIn(title_bars), LaggedStart(*[FadeIn(b) for b in bars], lag_ratio=0.3), FadeIn(*labels_bars), run_time=1.0)
        
        # Show cumulative number for top 2
        cum_text = MathTex(fr"\text{{Cumulative (PC1+PC2)}}={cum_explained[1]*100:.1f}\%").scale(0.55)
        cum_text.next_to(title_bars, DOWN, buff=0.15)
        self.add_fixed_in_frame_mobjects(cum_text)
        self.play(Write(cum_text))
        self.wait(0.8)

        # ----------------- PROJECTION & RECONSTRUCTION for k = 1..3 -----------------
        def project_to_k(points_, mean_, eigvecs_, k_):
            Qk = eigvecs_[:, :k_]
            centered_ = points_ - mean_
            Ys = (Qk.T @ centered_.T).T       # shape N x k (projected coordinates)
            Xhat = (Qk @ Ys.T).T + mean_      # shape N x 3 (reconstruction)
            return Xhat, Ys

        # Setup for projection loop
        recon_dots_group = VGroup()
        connector_lines_group = VGroup()
        sample_idx = 3 # Choose a sample point to highlight
        sample_pt = points[sample_idx]
        sample_dot = dots[sample_idx].copy().set_color(TEAL).scale(1.5)
        
        sample_label_text = MathTex(r"x_i").scale(0.7).to_corner(LEFT).shift(DOWN*0.1)
        sample_label_coords = MathTex(f"=({sample_pt[0]:.2f},{sample_pt[1]:.2f},{sample_pt[2]:.2f})").scale(0.55).next_to(sample_label_text, RIGHT, buff=0.1)
        self.add_fixed_in_frame_mobjects(sample_label_text, sample_label_coords)
        self.play(Transform(dots[sample_idx], sample_dot), Write(sample_label_text), Write(sample_label_coords))
        self.wait(0.4)

        residual_texts = VGroup()
        
        for k in [1, 2, 3]:
            # Compute reconstruction and projected coordinates
            Xhat, Ys = project_to_k(points, mean, eigvecs, k)
            recon_dots = VGroup(*[Dot3D(point=pt, color=ORANGE, radius=0.07) for pt in Xhat])
            connectors = VGroup(*[Line3D(start=points[i], end=Xhat[i], color=GRAY) for i in range(len(points))])
            
            # Show projection visual
            k_label = Text(f"Projection to k={k} PCs").scale(0.7).to_corner(UP).shift(DOWN*0.5)
            self.add_fixed_in_frame_mobjects(k_label)
            self.play(Write(k_label))
            self.play(LaggedStart(*[Create(c) for c in connectors], lag_ratio=0.01), run_time=0.9)
            self.play(LaggedStart(*[Create(rd) for rd in recon_dots], lag_ratio=0.01), run_time=0.6)
            
            # Numeric walkthrough for the highlighted sample point
            sample_proj = Xhat[sample_idx]
            coord = Ys[sample_idx]
            
            # Reconstruction formula
            recon_label = MathTex(
                rf"\hat{{x}} = \bar{{x}} + Q_{{{k}}} Y"
            ).scale(0.5).to_corner(LEFT).shift(DOWN*0.8)
            
            # Projected coordinates (Y)
            numeric_label = MathTex(
                rf"Y = Q_{{{k}}}^T(x-\bar{{x}})=\;(" + ", ".join([f"{v:.3f}" for v in coord]) + ")"
            ).scale(0.45).to_corner(LEFT).shift(DOWN*1.25)
            
            self.add_fixed_in_frame_mobjects(recon_label, numeric_label)
            self.play(Write(recon_label), Write(numeric_label))
            
            # Highlight the sample's reconstruction
            highlight = Dot3D(point=sample_proj, color=YELLOW, radius=0.12)
            self.play(FadeIn(highlight), run_time=0.5)
            self.wait(0.6)

            # Compute residual variance (Reconstruction Error)
            residual_var_k = np.mean(np.linalg.norm(points - Xhat, axis=1)**2)
            res_text = MathTex(f"k={k}\\;\\text{{Residual MSE}}={residual_var_k:.4f}", color=ORANGE).scale(0.5)
            res_text.to_corner(DL).shift(UP*(0.2*k))
            self.add_fixed_in_frame_mobjects(res_text)
            self.play(Write(res_text))
            residual_texts.add(res_text)
            self.wait(0.8)

            # Cleanup for next k
            self.play(FadeOut(k_label), FadeOut(recon_dots), FadeOut(connectors), FadeOut(highlight))
            self.play(FadeOut(recon_label), FadeOut(numeric_label))
            
        # Re-display all residual numbers for comparison
        self.play(FadeIn(residual_texts), run_time=0.6)
        self.wait(1.0)
        
        # Cleanup sample labels
        self.play(FadeOut(sample_label_text, sample_label_coords, dots[sample_idx]))
        self.wait(0.5)


        # ----------------- CAMERA FOCUS: ZOOM into mean & plane (k=2) -----------------
        # Hide the residuals and bar chart temporarily for a better view
        self.play(*[FadeOut(m) for m in [title_bars, cum_text, *bars, *labels_bars, *residual_texts, std_note, cov_matrix, cov_label, eqn, q_disp, lambda_disp, qT_disp, q_label, lambda_label, qT_label]], run_time=0.8)
        
        # Zoom camera onto the central area
        self.play(self.camera.frame.animate.set(width=6).move_to(mean), run_time=1.0)
        
        # Show the optimal projection plane (PC1 & PC2)
        v1, v2 = eigvecs[:, 0], eigvecs[:, 1]
        plane_size = 3.2
        corners = [mean +  v1*plane_size +  v2*plane_size,
                   mean +  v1*plane_size - v2*plane_size,
                   mean -  v1*plane_size - v2*plane_size,
                   mean -  v1*plane_size + v2*plane_size]
        plane = Polygon(*corners, color=YELLOW, fill_opacity=0.25, stroke_opacity=1.0)
        plane_label = MathTex(r"\text{Projection Plane (PC1, PC2)}").scale(0.6).to_corner(UL).shift(DOWN*0.1)
        
        self.add_fixed_in_frame_mobjects(plane_label)
        self.play(FadeIn(plane), Write(plane_label), run_time=0.8)
        self.wait(0.6)

        # Highlight orthogonality: show dot products of PC vectors ~ 0
        ortho_texts = VGroup()
        ortho_title = MathTex(r"\text{Principal Components are Orthogonal}").scale(0.5).to_corner(DR).shift(UP*1.0)
        self.add_fixed_in_frame_mobjects(ortho_title)
        self.play(Write(ortho_title))
        
        for i in range(3):
            for j in range(i+1,3):
                val = np.dot(eigvecs[:,i], eigvecs[:,j])
                tx = MathTex(f"v_{i+1} \\cdot v_{j+1} = {val:.3e}").scale(0.45)
                tx.next_to(ortho_title, DOWN).shift(DOWN*(0.18*(j-i)))
                self.add_fixed_in_frame_mobjects(tx)
                ortho_texts.add(tx)
                self.play(Write(tx), run_time=0.2)
        self.wait(1.0)

        # ----------------- FINISH: ambient rotate and final note -----------------
        # Final formula recap
        final_note = MathTex(r"\text{Final Goal: Dimensionality Reduction } (k < D)").scale(0.6)
        final_note.to_corner(UL).shift(RIGHT*1.0)
        self.add_fixed_in_frame_mobjects(final_note)
        self.play(Write(final_note))
        
        # Rotate the camera for a final view
        self.begin_ambient_camera_rotation(rate=0.02)
        self.wait(4.0)
        self.stop_ambient_camera_rotation()
        self.wait(0.5)

# To render the scene, run:
%manim -ql -v ERROR PCADeepDiveScene

Manim Community v0.19.0

NameError: cannot access free variable 'v' where it is not associated with a value in enclosing scope